## 와인 등급 예측 데이터셋
- 회귀 (등급을 맞추는 것이기 때문): 정확도보다는, RMSE, MAE 값으로 모델 성능 평가 진행 


In [1]:
# In[1]: Import statements should include all necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import mlflow
import mlflow.sklearn
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

df = pd.read_csv('winequality.csv') 
df


,type,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,white,7.0,0.270,0.36,20.7,0.045,45.0,170.0,1.00100,3.00,0.45,8.8,6
1,white,6.3,0.300,0.34,1.6,0.049,14.0,132.0,0.99400,3.30,0.49,9.5,6
2,white,8.1,0.280,0.40,6.9,0.050,30.0,97.0,0.99510,3.26,0.44,10.1,6
3,white,7.2,0.230,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
4,white,7.2,0.230,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6492,red,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5
6493,red,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,NaN,11.2,6
6494,red,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6
6495,red,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,5


In [2]:
df.info()
df.isna().sum()
df.describe() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6497 entries, 0 to 6496
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   type                  6497 non-null   object 
 1   fixed acidity         6487 non-null   float64
 2   volatile acidity      6489 non-null   float64
 3   citric acid           6494 non-null   float64
 4   residual sugar        6495 non-null   float64
 5   chlorides             6495 non-null   float64
 6   free sulfur dioxide   6497 non-null   float64
 7   total sulfur dioxide  6497 non-null   float64
 8   density               6497 non-null   float64
 9   pH                    6488 non-null   float64
 10  sulphates             6493 non-null   float64
 11  alcohol               6497 non-null   float64
 12  quality               6497 non-null   int64  
dtypes: float64(11), int64(1), object(1)
memory usage: 660.0+ KB


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
count,6487.000000,6489.000000,6494.000000,6495.000000,6495.000000,6497.000000,6497.000000,6497.000000,6488.000000,6493.000000,6497.000000,6497.000000
mean,7.216579,0.339691,0.318722,5.444326,0.056042,30.525319,115.744574,0.994697,3.218395,0.531215,10.491801,5.818378
std,1.296750,0.164649,0.145265,4.758125,0.035036,17.749400,56.521855,0.002999,0.160748,0.148814,1.192712,0.873255
min,3.800000,0.080000,0.000000,0.600000,0.009000,1.000000,6.000000,0.987110,2.720000,0.220000,8.000000,3.000000
25%,6.400000,0.230000,0.250000,1.800000,0.038000,17.000000,77.000000,0.992340,3.110000,0.430000,9.500000,5.000000
50%,7.000000,0.290000,0.310000,3.000000,0.047000,29.000000,118.000000,0.994890,3.210000,0.510000,10.300000,6.000000
75%,7.700000,0.400000,0.390000,8.100000,0.065000,41.000000,156.000000,0.996990,3.320000,0.600000,11.300000,6.000000
max,15.900000,1.580000,1.660000,65.800000,0.611000,289.000000,440.000000,1.038980,4.010000,2.000000,14.900000,9.000000


In [3]:
# 해당 가게의 와인은 등급별로 몇개가 있나요?
df['quality'].value_counts()
df.groupby('quality').size()

# 화이트/레드 와인별 와인의 등급은 몇 개씩 있나요?
df.groupby('type')['quality'].value_counts()
df.groupby(['type','quality']).size()

# 화이트/레드 와인의 평균 당도는 어느 정도 차이가 나나요?
df.groupby('type')['residual sugar'].mean()


# 와인의 등급에 가장 큰 영향을 미치는 top3 재료는 무엇인가요? 
df.corr(numeric_only=True)['quality'].sort_values(ascending=False)
# SQL,파있너 => GPT가 다 잘해주는데? 

quality                 1.000000
alcohol                 0.444319
citric acid             0.085706
free sulfur dioxide     0.055463
sulphates               0.038729
pH                      0.019366
residual sugar         -0.036825
total sulfur dioxide   -0.041385
fixed acidity          -0.077031
chlorides              -0.200886
volatile acidity       -0.265953
density                -0.305858
Name: quality, dtype: float64

In [4]:
# feature engineering

df=df.dropna(how='any')
df.isna().sum()


type                    0
fixed acidity           0
volatile acidity        0
citric acid             0
residual sugar          0
chlorides               0
free sulfur dioxide     0
total sulfur dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
quality                 0
dtype: int64

### Modeling

In [5]:
# (1) 데이터 분리

from sklearn.model_selection import train_test_split


train, test = train_test_split(df,test_size=0.3)

train.to_csv('wine_train.csv')
test.to_csv('wine_test.csv')

In [6]:
# (2) 데이터로드 (학습 테스트) =피쳐 데이터, 레이블 데이터 

X_train =  train.drop(columns=['quality', 'type'], axis=1)
X_test = test.drop(columns=['quality', 'type'], axis=1)

y_train = train['quality']
y_test = test['quality']


X_train.shape, y_train.shape
X_test.shape, y_test.shape

((1939, 11), (1939,))

In [7]:
# (3) 모델 학습  (mlflow load)

import mlflow
import mlflow.sklearn
from sklearn.linear_model import ElasticNet # 와인 데이터와 같은 다차원 데이터
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score # 수정
from itertools import product
mlflow.autolog()

mlflow.set_tracking_uri('http://127.0.0.1:5000')
mlflow.set_experiment(experiment_name='winequality_experiment')


2024/11/18 19:22:29 WARNING mlflow.utils.autologging_utils: MLflow sklearn autologging is known to be compatible with 0.24.1 <= scikit-learn <= 1.5.1, but the installed version is 1.5.2. If you encounter errors during autologging, try upgrading / downgrading scikit-learn to a compatible version, or try upgrading MLflow.
2024/11/18 19:22:30 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2024/11/18 19:22:30 INFO mlflow.tracking.fluent: Experiment with name 'winequality_experiment' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/472700017867975302', creation_time=1731925350367, experiment_id='472700017867975302', last_update_time=1731925350367, lifecycle_stage='active', name='winequality_experiment', tags={}>

In [8]:
# 실험 설계
from sklearn.linear_model import ElasticNet
from itertools import product
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

alpha = [0.2, 0.5, 0.7, 1.0]
l1_ratio = [0.2, 0.3, 0.7, 1.0]

mlflow.autolog()

for a, l in product(alpha, l1_ratio):
    with mlflow.start_run(nested=True):
        lr = ElasticNet(alpha=a, l1_ratio=l, random_state=123)
        lr.fit(X_train, y_train) # 모의고사 문제, 모의고사 정답 => 공부좀 하고있어

        pred = lr.predict(X_test) # 수능 문제

        # 모델 성능 평가
        rmse = np.sqrt(mean_squared_error(y_test, pred)) # RMSE
        mae = mean_absolute_error(y_test, pred) # MAE
        r2 = r2_score(y_test, pred)

        # 수동으로 log 기록
        mlflow.log_params({'alpha': a, 'l1_ratio': l})
        mlflow.log_metrics({'rmse': rmse, 'mae': mae, 'r2': r2})
        
        mlflow.sklearn.log_model(lr, 'winequality_model')

        print(f"Alpha: {a}, L1 ratio: {l}, RMSE: {rmse:.4f}")

2024/11/18 19:24:05 WARNING mlflow.utils.autologging_utils: MLflow sklearn autologging is known to be compatible with 0.24.1 <= scikit-learn <= 1.5.1, but the installed version is 1.5.2. If you encounter errors during autologging, try upgrading / downgrading scikit-learn to a compatible version, or try upgrading MLflow.
2024/11/18 19:24:05 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2024/11/18 19:24:08 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2024/11/18 19:24:08 INFO mlflow.tracking._tracking_service.client: 🏃 View run mysterious-chimp-822 at: http://127.0.0.1:5000/#/experiments/472700017867975302/runs/06ea4701b2ba451daf4011f919e2d537.
2024/11/18 19:24:08 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/472700017867975302.


Alpha: 0.2, L1 ratio: 0.2, RMSE: 0.7792


2024/11/18 19:24:10 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2024/11/18 19:24:10 INFO mlflow.tracking._tracking_service.client: 🏃 View run unleashed-panda-34 at: http://127.0.0.1:5000/#/experiments/472700017867975302/runs/30568fd8f5b842b5966e9cc37c0d6bf4.
2024/11/18 19:24:10 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/472700017867975302.


Alpha: 0.2, L1 ratio: 0.3, RMSE: 0.7806


2024/11/18 19:24:12 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2024/11/18 19:24:12 INFO mlflow.tracking._tracking_service.client: 🏃 View run powerful-roo-424 at: http://127.0.0.1:5000/#/experiments/472700017867975302/runs/ceae13837d924625a1a137253c975a3b.
2024/11/18 19:24:12 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/472700017867975302.


...

Alpha: 1.0, L1 ratio: 0.3, RMSE: 0.8362


2024/11/18 19:24:33 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2024/11/18 19:24:33 INFO mlflow.tracking._tracking_service.client: 🏃 View run secretive-fish-613 at: http://127.0.0.1:5000/#/experiments/472700017867975302/runs/c42bd308517a41f39ceae014e16f1ddb.
2024/11/18 19:24:33 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/472700017867975302.


Alpha: 1.0, L1 ratio: 0.7, RMSE: 0.8753


2024/11/18 19:24:35 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2024/11/18 19:24:35 INFO mlflow.tracking._tracking_service.client: 🏃 View run likeable-toad-253 at: http://127.0.0.1:5000/#/experiments/472700017867975302/runs/76f45e9a09304314bc70890e5178ac54.
2024/11/18 19:24:35 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/472700017867975302.


Alpha: 1.0, L1 ratio: 1.0, RMSE: 0.8792


In [88]:
# (4) 모델 서빙

# http://127.0.0.1:5000/#/experiments/472700017867975302/runs/76f45e9a09304314bc70890e5178ac54

# mlflow models serve -m ./mlartifacts/472700017867975302/76f45e9a09304314bc70890e5178ac54/artifacts/model -p 5002 --no-conda

SyntaxError: invalid syntax (2048475131.py, line 5)

In [25]:
import requests
import json

test_data = pd.read_csv('wine_test.csv', index_col=0)
input_data = test_data.drop(['quality'], axis=1)[:3]

data = {
    'dataframe_split': input_data.to_dict(orient='split')
}

url = 'http://127.0.0.1:5002/invocations'

headers = {'Content-Type': 'application/json'}
res = requests.post(url, headers=headers, data=json.dumps(data))

result = res.json()
print("result: ", result)


result:  {'predictions': [5.80301347737781, 5.828560670861267, 5.846929314109703]}
